In [ ]:
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
#need to add in qc and mean
# still need to add pre synaptic spike number qc. ie.e does more than one presynaptic spike happen?
# what if a postsynaptic pulse happens and it does not pass qc 

In [ ]:
def get_data_from_csv(filename):
    data = pd.read_csv(filename)
    d={}
    d['amps'] = []
    d['spike_times_relative_to_expt'] = []
    d['post_syn_rec_id'] = []
    d['expt_start_time'] = []
    d['post_id'] = data['post_id']
    d['pre_id'] = data['pre_id']
    d['uid'] = data['uid']
    d['stim_freq'] = []
    d['pulse_num_within_train'] = []
#    d['qc_mask'] = []
    
    #convert weird string from cvs to arrays
    for ii in range(len(data)):
        d['amps'].append(np.fromstring(data['deconvolved_amps'][ii].replace('\n', '').replace('[', '').replace(']', ''), dtype=float, sep=' '))
        d['spike_times_relative_to_expt'].append(np.fromstring(data['spike_times_relative_to_experiment'][ii].replace('\n', '').replace('[', '').replace(']', ''), dtype=float, sep=' '))
        d['expt_start_time'].append(d['spike_times_relative_to_expt'][-1][0])
        d['post_syn_rec_id'].append(np.fromstring(data['post_syn_rec_id'][ii].replace('\n', '').replace('[', '').replace(']', ''), dtype=int, sep=' '))
        d['pulse_num_within_train'].append(np.fromstring(data['post_syn_rec_id'][ii].replace('\n', '').replace('[', '').replace(']', ''), dtype=int, sep=' '))
        #print(data['stim_freq'][ii])
        d['stim_freq'].append(np.fromstring(data['stim_freq'][ii].replace('\n', '').replace('[', '').replace(']', ''), dtype=float, sep=' '))
        if (len(d['amps'][-1]) != len(d['spike_times_relative_to_expt'][-1])) or \
            (len(d['amps'][-1]) != len(d['post_syn_rec_id'][-1])) or \
            (len(d['amps'][-1]) != len(d['stim_freq'][-1])) or \
            (len(d['amps'][-1]) != len(d['pulse_num_within_train'][-1])):
            raise Exception('string to array conversion probably went wrong')
    return d

filename='pv_tlx_to_sst.csv'
data=get_data_from_csv(filename)

In [ ]:
for ii in range(len(data)):
    plt.figure(figsize=(15,10))
    
    # plot whole experiment
    sample=range(0,len(data['amps']))
    plt.plot(data['spike_times_relative_to_expt'][ii][sample], data['amps'][ii][sample], '.-', ms=20)
    # for ii, (p,t,a) in enumerate(zip(pulse_ids[sample], spike_times_relative_to_expt[sample], amplitudes[sample]*1e3)):
    #     plt.annotate(str(ii), xy=(t,a), textcoords='data')
    plt.ylabel('Deconvolution Arbitrary Units')
    plt.xlabel('time (unknown units)')
    plt.title('Deconvolution Amplitude\n%.3f, pre %i, post %i' %(data['uid'][ii], data['pre_id'][ii], data['post_id'][ii]))
    
    # differentiate individual trains and put them in a dictionary for dataFrame
    conn_dict={}
    conn_dict['train_num'] = []
    conn_dict['post_rec_id'] = []
    conn_dict['amps'] = []
    conn_dict['times_rel_to_expt'] = []
    conn_dict['times_rel_to_train'] = []
    conn_dict['pulse_num_within_train'] = [] 
    conn_dict['stim_freq'] = []

    plt.figure(figsize=(15,10))
    start_train_index = 0
    end_train_index = 0
    train_num = 1
    train_id = data['post_syn_rec_id'][ii]
    for jj in range(1, len(train_id)):
        if (train_id[jj-1] != train_id[jj]):
            end_train_index = jj-1  
            #make the plot
            sample = range(start_train_index, end_train_index+1) #+1 due to range
            train_stim_freq=np.unique(data['stim_freq'][ii][sample])
            rec_id=np.unique(data['post_syn_rec_id'][ii][sample])

            if len(train_stim_freq) != 1:
                raise Exception('train should have one freq')
            
            train_times_rel_to_expt = data['spike_times_relative_to_expt'][ii][sample]
            train_times_rel_to_train = data['spike_times_relative_to_expt'][ii][sample]-data['spike_times_relative_to_expt'][ii][start_train_index]
            train_amps = data['amps'][ii][sample]
            pulse_num_within_train = data['pulse_num_within_train'][ii][sample]
            
            plt.plot(train_times_rel_to_train, train_amps, '.-', ms=20)
            start_train_index = jj
    
            #put in dictionary
            conn_dict['train_num'].append(train_num)
            conn_dict['post_rec_id'].append(rec_id)
            conn_dict['amps'].append(train_amps)
            conn_dict['times_rel_to_expt'].append(train_times_rel_to_expt)
            conn_dict['times_rel_to_train'].append(train_times_rel_to_train)
            conn_dict['pulse_num_within_train'].append(pulse_num_within_train)
            conn_dict['stim_freq'].append(train_stim_freq)

            train_num += 1
#        put into new data frame and organize by stim freq
    plt.show()

In [ ]:
conn_dict.keys()

In [ ]:
conn_dict['stim_freq']